<a href="https://colab.research.google.com/github/JessMog/Google---Fast-or-Slow-Predict-AI-Model-Runtime/blob/main/AI_Model_Runtime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_gnn --pre
!pip install tensorflow_ranking

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.4/838.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.6/761.6 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.7 MB/s e

In [ ]:
!pip install tpugraphsv1_layout_data_py
!pip install tpugraphsv1_tile_data_py
!pip install tpugraphsv1_implicit_py

In [6]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_gnn as tfgnn
import tensorflow_ranking as tfr

## Training Pipelines
The following code is organized as:

Helper functions: MLP (_mlp) and Embedding layer (_Opembedding). The embedding layer amends a feature on the op nodes, with name op_e, by embedding the integral op IDs.
Pipeline code for training on the Layout collections.
Pipeline code for training on the Tile collection.

### Helper functions, for both Layout and Tile collections.

In [11]:
def _mlp(dims, hidden_activation, l2reg=1e-4, use_bias=True):
  """Helper function for multi-layer perceptron (MLP)."""
  layers = []
  for i, dim in enumerate(dims):
    if i > 0:
      layers.append(tf.keras.layers.Activation(hidden_activation))
    layers.append(tf.keras.layers.Dense(
        dim, kernel_regularizer=tf.keras.regularizers.l2(l2reg),
        use_bias=use_bias))
  return tf.keras.Sequential(layers)


class _OpEmbedding(tf.keras.Model):
  """Embeds GraphTensor.node_sets['op']['op'] nodes into feature 'op_e'."""

  def __init__(self, num_ops: int, embed_d: int, l2reg: float = 1e-4):
    super().__init__()
    self.embedding_layer = tf.keras.layers.Embedding(
        num_ops, embed_d, activity_regularizer=tf.keras.regularizers.l2(l2reg))

  def call(
      self, graph: tfgnn.GraphTensor,
      training: bool = False) -> tfgnn.GraphTensor:
    op_features = dict(graph.node_sets['op'].features)
    op_features['op_e'] = self.embedding_layer(
        tf.cast(graph.node_sets['op']['op'], tf.int32))
    return graph.replace_features(node_sets={'op': op_features})

## Layout Training Pipeline
We start by defining constants:

`Batch sizes = num graphs`, num sampled nodes per graph, and num configurations per graph.
Collection to train on: source `(xla versus nlp)` and search stragey `(random versus default)`.
Then, boilerplate code to prepare the datasets.

Then, we dive deeper into the dataset examples `(a batch of graphs from the tiles collection)`.

Finally, details on defining a model.


---



Define constants and choose subcollection
We load BATCH_SIZE graphs per batch. Each will have

In [12]:
LAYOUT_DATA_ROOT = '/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout'
SOURCE = 'xla'  # Can be "xla" or "nlp"
SEARCH = 'random'  # Can be "random" or "default"

# Batch size information.
BATCH_SIZE = 16  # Number of graphs per batch.
CONFIGS_PER_GRAPH = 5  # Number of configurations (features and target values) per graph.
MAX_KEEP_NODES = 1000  # Useful for dropout.
# `MAX_KEEP_NODES` is (or, is not) useful for Segment Dropout, if model uses
# edges "sampled_config" and "sampled_feed" (or, "config" and "feed")

In [ ]:
layout_data_root_dir = os.path.join(
      os.path.expanduser(LAYOUT_DATA_ROOT), SOURCE, SEARCH)

layout_npz_dataset = layout_data.get_npz_dataset(
    layout_data_root_dir,
    min_train_configs=CONFIGS_PER_GRAPH,
    max_train_configs=500,  # If any graph has more than this configurations, it will be filtered [speeds up loading + training]
    cache_dir='cache'
)

def pair_layout_graph_with_label(graph: tfgnn.GraphTensor):
    """Extracts label from graph (`tfgnn.GraphTensor`) and returns a pair of `(graph, label)`"""
    # Return runtimes divded over large number: only ranking is required. The
    # runtimes are in the 100K range
    label = tf.cast(graph.node_sets['g']['runtimes'], tf.float32) / 1e7
    return graph, label

layout_train_ds = (
      layout_npz_dataset.train.get_graph_tensors_dataset(
          CONFIGS_PER_GRAPH, max_nodes=MAX_KEEP_NODES)
      .shuffle(100, reshuffle_each_iteration=True)
      .batch(BATCH_SIZE, drop_remainder=False)
      .map(tfgnn.GraphTensor.merge_batch_to_components)
      .map(pair_layout_graph_with_label))

layout_valid_ds = (
      layout_npz_dataset.validation.get_graph_tensors_dataset(
          CONFIGS_PER_GRAPH)
      .batch(BATCH_SIZE, drop_remainder=False)
      .map(tfgnn.GraphTensor.merge_batch_to_components)
      .map(pair_layout_graph_with_label))

**Prepare** `tf.data.Dataset` instances
Specifically, `layout_train_ds` and `layout_valid_ds`.


---



It can take ~10 minutes .

In [ ]:
layout_data_root_dir = os.path.join(
      os.path.expanduser(LAYOUT_DATA_ROOT), SOURCE, SEARCH)

layout_npz_dataset = layout_data.get_npz_dataset(
    layout_data_root_dir,
    min_train_configs=CONFIGS_PER_GRAPH,
    max_train_configs=500,  # If any graph has more than this configurations, it will be filtered [speeds up loading + training]
    cache_dir='cache'
)

def pair_layout_graph_with_label(graph: tfgnn.GraphTensor):
    """Extracts label from graph (`tfgnn.GraphTensor`) and returns a pair of `(graph, label)`"""
    # Return runtimes divded over large number: only ranking is required. The
    # runtimes are in the 100K range
    label = tf.cast(graph.node_sets['g']['runtimes'], tf.float32) / 1e7
    return graph, label

layout_train_ds = (
      layout_npz_dataset.train.get_graph_tensors_dataset(
          CONFIGS_PER_GRAPH, max_nodes=MAX_KEEP_NODES)
      .shuffle(100, reshuffle_each_iteration=True)
      .batch(BATCH_SIZE, drop_remainder=False)
      .map(tfgnn.GraphTensor.merge_batch_to_components)
      .map(pair_layout_graph_with_label))

layout_valid_ds = (
      layout_npz_dataset.validation.get_graph_tensors_dataset(
          CONFIGS_PER_GRAPH)
      .batch(BATCH_SIZE, drop_remainder=False)
      .map(tfgnn.GraphTensor.merge_batch_to_components)
      .map(pair_layout_graph_with_label))

dataset cache file:  cache/317e146d60640edb255d1dcb2d3235c8-cache.npz
100%|██████████| 69/69 [01:03<00:00,  1.09it/s]
Saving ...
wrote cache/317e146d60640edb255d1dcb2d3235c8-cache.npz
wrote cache/317e146d60640edb255d1dcb2d3235c8-cache.npz.graphs.txt
dataset cache file:  cache/ed1b35ba5a151cd92dc0e4d41f4160ce-cache.npz
100%|██████████| 7/7 [00:07<00:00,  1.07s/it]
Saving ...
wrote cache/ed1b35ba5a151cd92dc0e4d41f4160ce-cache.npz
wrote cache/ed1b35ba5a151cd92dc0e4d41f4160ce-cache.npz.graphs.txt
dataset cache file:  cache/437403654dcd7a1b5d98d25edfb15ce6-cache.npz
100%|██████████| 8/8 [00:01<00:00,  5.48it/s]
Saving ...
wrote cache/437403654dcd7a1b5d98d25edfb15ce6-cache.npz
wrote cache/437403654dcd7a1b5d98d25edfb15ce6-cache.npz.graphs.txt